# Enhanced Image Classification with Item Data

This notebook tests the hierarchical classification system on test_df.csv items using both images and item metadata (supplier_name, supplier_reference_description, materials).

## Setup
1. Make sure you have set your OpenRouter API key in `.env` file:
   ```
   OPENROUTER_API_KEY=your_api_key_here
   ```
2. The model is configured as `google/gemma-3-27b-it:free` in `app/config/settings.py`
3. **Note**: This notebook will classify items even when images are not found, using metadata only.


In [ ]:
import sys
import os
from pathlib import Path
import pandas as pd
import base64
from tqdm import tqdm
import time

# Add parent directory to path
sys.path.append(str(Path.cwd().parent))

from app.langchain_modules.graph.classificationGraph import ClassificationGraphBuilder
from app.langchain_modules.llm_definitions.openrouter_client import create_openrouter_client


In [ ]:
# Configuration
TEST_DF_PATH = "../resources/test_df.csv"
PICS_AW_DIR = "../pics/2026_AW"
PICS_SS_DIR = "../pics/2026_SS"

# Load test dataframe
test_df = pd.read_csv(TEST_DF_PATH)
print(f"Loaded {len(test_df)} items from test_df.csv")
print(f"Columns: {test_df.columns.tolist()}")


In [ ]:
def find_image_for_item(item_id: int, pics_aw_dir: Path, pics_ss_dir: Path) -> Path | None:
    """
    Find image file for a given item_id.
    Searches in both 2026_AW and 2026_SS directories.
    
    Args:
        item_id: Item ID to search for
        pics_aw_dir: Path to 2026_AW directory
        pics_ss_dir: Path to 2026_SS directory
    
    Returns:
        Path to image file if found, None otherwise
    """
    item_id_str = str(item_id)
    
    # Search in 2026_AW
    for ext in ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']:
        pattern = f"{item_id_str}*{ext}"
        matches = list(pics_aw_dir.rglob(pattern))
        if matches:
            # Return first match (prefer non-thumbnail if available)
            non_thumbnails = [m for m in matches if 'thumbnail' not in str(m).lower()]
            return non_thumbnails[0] if non_thumbnails else matches[0]
    
    # Search in 2026_SS
    for ext in ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']:
        pattern = f"{item_id_str}*{ext}"
        matches = list(pics_ss_dir.rglob(pattern))
        if matches:
            # Return first match (prefer non-thumbnail if available)
            non_thumbnails = [m for m in matches if 'thumbnail' not in str(m).lower()]
            return non_thumbnails[0] if non_thumbnails else matches[0]
    
    return None


In [ ]:
def image_to_base64(image_path: Path) -> str:
    """
    Convert image file to base64 data URI.
    
    Args:
        image_path: Path to image file
    
    Returns:
        Base64 data URI string
    """
    with open(image_path, 'rb') as f:
        image_data = base64.b64encode(f.read()).decode('utf-8')
    
    # Determine MIME type from extension
    ext = image_path.suffix.lower()
    mime_types = {
        '.jpg': 'image/jpeg',
        '.jpeg': 'image/jpeg',
        '.png': 'image/png'
    }
    mime_type = mime_types.get(ext, 'image/jpeg')
    
    return f"data:{mime_type};base64,{image_data}"


In [ ]:
def extract_item_data(row: pd.Series) -> tuple:
    """
    Extract item metadata from dataframe row.
    
    Args:
        row: DataFrame row
    
    Returns:
        Tuple of (supplier_name, supplier_reference_description, materials)
        Returns None for missing values
    """
    supplier_name = row.get('supplier_name')
    supplier_reference_description = row.get('supplier_reference_description')
    materials = row.get('materials')
    
    # Convert to None if NaN or empty string
    supplier_name = None if pd.isna(supplier_name) or not str(supplier_name).strip() else str(supplier_name).strip()
    supplier_reference_description = None if pd.isna(supplier_reference_description) or not str(supplier_reference_description).strip() else str(supplier_reference_description).strip()
    materials = None if pd.isna(materials) or not str(materials).strip() else str(materials).strip()
    
    return supplier_name, supplier_reference_description, materials


In [ ]:
# Initialize classification system
print("Initializing classification system...")
llm = create_openrouter_client()
graph_builder = ClassificationGraphBuilder(llm)
graph_builder.build()
print("Classification system ready!")


In [ ]:
# Prepare paths
pics_aw_path = Path(PICS_AW_DIR)
pics_ss_path = Path(PICS_SS_DIR)

# Configuration for incremental saving
CHECKPOINT_FILE = "../resources/test_df_with_data_checkpoint.csv"
FINAL_OUTPUT_FILE = "../resources/test_df_with_data_predictions.csv"
SAVE_EVERY_N_ITEMS = 5  # Save every 5 items processed

# Check if checkpoint exists and load it
checkpoint_path = Path(CHECKPOINT_FILE)
RESUME_FROM_CHECKPOINT = True  # Set to False to start fresh

if checkpoint_path.exists() and RESUME_FROM_CHECKPOINT:
    print(f"✓ Found checkpoint file: {CHECKPOINT_FILE}")
    print("Loading checkpoint to resume from previous run...")
    result_df = pd.read_csv(checkpoint_path)
    already_processed = result_df['predicted_main'].notna().sum()
    print(f"✓ Loaded {len(result_df)} items from checkpoint")
    print(f"✓ Items already processed: {already_processed}")
    print(f"✓ Remaining items: {len(result_df) - already_processed}")
    print("  (Set RESUME_FROM_CHECKPOINT = False in the cell above to start fresh)")
else:
    # Create a copy of test_df for results
    result_df = test_df.copy()
    
    # Add new columns for classification results (use object dtype to avoid warnings)
    result_df['predicted_main'] = None
    result_df['predicted_sub'] = None
    result_df['predicted_detail'] = None
    result_df['predicted_level4'] = None
    result_df['classification_errors'] = None
    result_df['image_found'] = False

# Convert to object dtype to avoid pandas warnings (only if columns exist)
for col in ['predicted_main', 'predicted_sub', 'predicted_detail', 'predicted_level4', 'classification_errors']:
    if col in result_df.columns:
        result_df[col] = result_df[col].astype('object')

print(f"\nResult dataframe ready with {len(result_df)} rows")
print(f"Incremental saving: Every {SAVE_EVERY_N_ITEMS} items")
print(f"Checkpoint file: {CHECKPOINT_FILE}")
print(f"Final output file: {FINAL_OUTPUT_FILE}")


In [ ]:
# Process each item with incremental saving
total_items = len(result_df)
successful = 0
failed = 0
no_image = 0
skipped = 0  # Items already processed

print(f"Processing {total_items} items...")
print("This may take a while depending on API rate limits.")
print(f"Results will be saved every {SAVE_EVERY_N_ITEMS} items to: {CHECKPOINT_FILE}")
print("Note: Items without images will be classified using metadata only.\n")

processed_count = 0

for idx, row in tqdm(result_df.iterrows(), total=total_items, desc="Classifying"):
    item_id = row['item_id']
    
    # Skip if already processed (has a prediction)
    if pd.notna(result_df.at[idx, 'predicted_main']):
        skipped += 1
        continue
    
    # Extract item data
    supplier_name, supplier_reference_description, materials = extract_item_data(row)
    
    # Find image for this item
    image_path = find_image_for_item(item_id, pics_aw_path, pics_ss_path)
    
    if image_path is None:
        result_df.at[idx, 'image_found'] = False
        no_image += 1
        image_base64 = None  # Will use metadata-only classification
    else:
        result_df.at[idx, 'image_found'] = True
        # Convert image to base64
        image_base64 = image_to_base64(image_path)
    
    # Always call classify() - with image if available, or metadata-only if not
    try:
        # Run classification with retry logic for rate limits
        max_retries = 5
        base_delay = 30  # Start with 30 seconds (more conservative)
        result = None
        last_error = None
        
        for attempt in range(max_retries):
            try:
                result = graph_builder.classify(
                    image_data=image_base64,
                    supplier_name=supplier_name,
                    supplier_reference_description=supplier_reference_description,
                    materials=materials
                )
                
                # Check if result has critical errors even if no exception was raised
                if result.errors:
                    error_str = '; '.join(result.errors)
                    is_rate_limit = '429' in error_str or 'rate limit' in error_str.lower() or 'rate-limited' in error_str.lower()
                    is_400_error = '400' in error_str or 'Developer instruction' in error_str
                    
                    # If critical error and not last attempt, retry
                    if (is_rate_limit or is_400_error) and attempt < max_retries - 1:
                        wait_time = base_delay * (2 ** attempt)
                        error_type = "Rate limit" if is_rate_limit else "System message"
                        tqdm.write(f"⚠️  {error_type} error in result for item {item_id}, waiting {wait_time}s before retry {attempt + 1}/{max_retries}")
                        time.sleep(wait_time)
                        result = None  # Reset to retry
                        continue
                    # If main classification failed, that's critical
                    elif not result.main and any('MainClassifier' in e for e in result.errors):
                        if attempt < max_retries - 1:
                            wait_time = base_delay * (2 ** attempt)
                            tqdm.write(f"⚠️  Main classification failed for item {item_id}, waiting {wait_time}s before retry {attempt + 1}/{max_retries}")
                            time.sleep(wait_time)
                            result = None
                            continue
                
                # If we got here and have a result (even with minor errors), break
                if result:
                    break
                    
            except Exception as e:
                last_error = e
                error_str = str(e)
                # Check if it's a rate limit error or 400 error (system message issue)
                is_rate_limit = '429' in error_str or 'rate limit' in error_str.lower() or 'rate-limited' in error_str.lower()
                is_400_error = '400' in error_str or 'Developer instruction' in error_str
                
                if (is_rate_limit or is_400_error) and attempt < max_retries - 1:
                    # Exponential backoff: 30s, 60s, 120s, 240s, 480s
                    wait_time = base_delay * (2 ** attempt)
                    error_type = "Rate limit" if is_rate_limit else "System message"
                    tqdm.write(f"⚠️  {error_type} error for item {item_id}, waiting {wait_time}s before retry {attempt + 1}/{max_retries}")
                    time.sleep(wait_time)
                    continue
                # If not retryable error or last attempt, break to handle error
                break
        
        if result and result.main:  # Only count as successful if we got at least main
            # Store results
            result_df.at[idx, 'predicted_main'] = result.main
            result_df.at[idx, 'predicted_sub'] = result.sub if result.sub else None
            result_df.at[idx, 'predicted_detail'] = result.detail if result.detail else None
            result_df.at[idx, 'predicted_level4'] = result.level4 if result.level4 else None
            
            if result.errors:
                result_df.at[idx, 'classification_errors'] = '; '.join(result.errors)
            
            successful += 1
        else:
            # Classification failed after all retries
            error_msg = str(last_error) if last_error else "Classification failed after retries"
            if result and result.errors:
                error_msg = '; '.join(result.errors)
            result_df.at[idx, 'classification_errors'] = error_msg
            failed += 1
            tqdm.write(f"❌ Failed after {max_retries} retries for item {item_id}")
        
        processed_count += 1
        
        # Add delay to respect rate limits (free tier: 16 requests/min = ~3.75s between requests)
        # But we're doing 4 API calls per item (main, sub, detail, level4), so need more delay
        # 16 requests/min = 1 request per 3.75s, but we do 4 requests = need ~15s between items
        time.sleep(15.0)  # More conservative delay for free tier (4 API calls per item)
        
    except Exception as e:
        result_df.at[idx, 'classification_errors'] = str(e)
        failed += 1
        processed_count += 1
        error_msg = str(e)
        if '429' in error_msg or 'rate limit' in error_msg.lower():
            tqdm.write(f"❌ Rate limit error for item {item_id}, will retry on next run")
        else:
            tqdm.write(f"❌ Error processing item {item_id}: {error_msg[:100]}")
    
    # Save checkpoint every N items
    if processed_count > 0 and processed_count % SAVE_EVERY_N_ITEMS == 0:
        result_df.to_csv(CHECKPOINT_FILE, index=False)
        tqdm.write(f"💾 Checkpoint saved: {processed_count} items processed, "
                  f"{successful} successful, {failed} failed, {no_image} no image")

# Final save
result_df.to_csv(CHECKPOINT_FILE, index=False)
result_df.to_csv(FINAL_OUTPUT_FILE, index=False)

print(f"\n\n{'='*60}")
print(f"Classification complete!")
print(f"{'='*60}")
print(f"Total items: {total_items}")
print(f"Successful: {successful}")
print(f"Failed: {failed}")
print(f"No image found (metadata-only): {no_image}")
print(f"Skipped (already processed): {skipped}")
print(f"\nResults saved to:")
print(f"  - Checkpoint: {CHECKPOINT_FILE}")
print(f"  - Final output: {FINAL_OUTPUT_FILE}")


## Helper: Clear errors for failed items

Run this cell if you want to retry items that failed. This will clear the `predicted_main` value for items that have errors, allowing them to be retried.


In [ ]:
# Clear errors for failed items to allow retry
if 'classification_errors' in result_df.columns:
    failed_mask = result_df['classification_errors'].notna()
    failed_count = failed_mask.sum()
    if failed_count > 0:
        print(f"Found {failed_count} items with errors.")
        result_df.loc[failed_mask, 'predicted_main'] = None
        result_df.loc[failed_mask, 'predicted_sub'] = None
        result_df.loc[failed_mask, 'predicted_detail'] = None
        result_df.loc[failed_mask, 'predicted_level4'] = None
        result_df.loc[failed_mask, 'classification_errors'] = None
        result_df.to_csv(CHECKPOINT_FILE, index=False)
        print(f"✓ Cleared predictions for {failed_count} items. They will be retried on next run.")
    else:
        print("No items with errors found.")
else:
    print("No classification_errors column found.")
